# IMPORTS

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Fetching File Name and Number of Clusters

In [ ]:
filePath = input("Please enter file name: ") #"cho.txt" 
k = int(input("Number of Clusters: "))

# Calculating the Distance Matrix

In [ ]:
file = open(filePath, "r")
lines = file.readlines()
df1 = ""

# splittng the line into individual data
flag1 = 2
for line in lines:
    if(" " in line):
        flag1 = 0
    elif("\t" in line):
        flag1 = 1
    break

if(flag1 == 0):
    for line in lines:
        df1 = pd.read_csv(filePath, sep = ' ', lineterminator = '\n', header = None)
elif(flag1 == 1):
    for line in lines:
        df1 = pd.read_csv(filePath, sep = '\t', lineterminator = '\n', header = None)
else:
    print("Data should be sperated by \t or single space!")
    
df2 = df1.copy()

(rownum,colnum) = (df2.shape)
distance_matrix = np.zeros((rownum, rownum))

for i in range(rownum):
    q1 = df2.iloc[i,2:].values
    for j in range(rownum):
        q2 = df2.iloc[j,2:].values
        distance_matrix[i][j] = np.linalg.norm(q1-q2)

# Hierarchical Agglomerative Clustering Implementation

In [ ]:
obvs = list(df2.iloc[:,0].values)
dist_mat = distance_matrix.copy()

#def hac(k,obvs,dist_mat,rownum,colnum)
while(len(obvs)> k):
    #min_dist = find_min(updist_mat, min_dist)
    pos1 = 0
    pos2 = 0
    min_dist = 1000
    for i in range(dist_mat.shape[0]):
        for j in range(dist_mat.shape[0]):
            if(dist_mat[i][j]==0):
                break
            elif(dist_mat[i][j]<min_dist):
                    min_dist = dist_mat[i][j]
                    (pos1,pos2) = (i,j)
    
    for i in range(dist_mat.shape[0]):        
        dt1 = dist_mat[pos1][i]
        dt2 = dist_mat[pos2][i]
        min_dist1 = dt1
        if(dt1 > dt2):
            min_dist1 = dt2
        else:
            min_dist1 = dt1
        dist_mat[pos1][i] = min_dist1
        
        dt3 = dist_mat[i][pos1]
        dt4 = dist_mat[i][pos2]
        min_dist2 = dt3
        if(dt3 > dt4):
            min_dist2 = dt4
        else:
            min_dist2 = dt3
        dist_mat[i][pos1] = min_dist2
        
        
    obvs[pos1] = list((obvs[pos1],obvs[pos2]))
    dist_mat = np.delete(dist_mat, pos2, axis = 0)
    dist_mat = np.delete(dist_mat, pos2, axis = 1)
        
    del obvs[pos2]
    
def listflatten(l):
    clustered_obvs = []
    if(type(l)==list):
        for i in l:
            clustered_obvs += listflatten(i) #recursively managing list within a list
    else:
        clustered_obvs.append(l)
    return clustered_obvs

clustered_obvs = []
for i in obvs:
    clustered_obvs.append(listflatten(i))

# Displaying the Clusters formed

In [ ]:
print(clustered_obvs)

# Jaccard Co-eff and Rand Index Calculation

In [ ]:
obvs_truth = []
for p1 in range(df2.shape[0]):
    obvs_truth.append(0)
cnt = 0    
for i in range(len(clustered_obvs)):
    for j in (clustered_obvs[i]):
        obvs_truth[j-1] = (i+1)
        
grd_truth = list(df2.iloc[:,1].values)
def cjr(ground_truth,observed_truth):
    true_pos = 0
    true_neg = 0
    false_pos=0
    false_neg=0
    for i in range(df2.shape[0]):
        for j in range(df2.shape[0]):
            if ground_truth[i]==ground_truth[j]:
                if observed_truth[i]==observed_truth[j]:
                    true_pos=true_pos+1
                else:
                    false_neg=false_neg+1
            elif ground_truth[i]!=ground_truth[j]:
                if observed_truth[i]==observed_truth[j]:
                    false_pos=false_pos+1
                else:
                    true_neg=true_neg+1
    jaccard_coeff = (true_pos)/(true_pos+false_pos+false_neg)
    rand_index = (true_pos+true_neg)/(true_pos+true_neg+false_pos+false_neg)
    return jaccard_coeff,rand_index

jaccard_coeff,rand_index = cjr(grd_truth,obvs_truth)

# Data Visualization and Displaying the Jaccard & Rand Index Values

In [ ]:
observed_truth_nparr = np.asarray(obvs_truth)
df3 = df2.copy()
X_df = df3.iloc[:,2:].values
ldf = pd.DataFrame(observed_truth_nparr,columns=["Label"])

#X_df = StandardScaler().fit_transform(X_df)
if(df2.shape[1]>4):
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(X_df)
    principalDf = pd.DataFrame(data = principalComponents,columns = ['PC1', 'PC2'])
    finalDf = pd.concat([principalDf, ldf["Label"]], axis = 1)
    print()
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    til1 = "HAC ("+str(filePath)+") with "+str(k)+" clusters"
    ax.set_title(til1, fontsize = 20)
    targets = list(set(observed_truth_nparr))
    for pt in targets:
        indicesToKeep = finalDf['Label'] == pt
        ax.scatter(finalDf.loc[indicesToKeep, 'PC1'], finalDf.loc[indicesToKeep, 'PC2'], s = 50)
    ax.legend(targets)
    ax.grid()

elif(df2.shape[1]==4):
    dfnew = df3.iloc[:,2:4]
    dfnew.columns = ['X','Y']
    finalDf = pd.concat([dfnew, ldf["Label"]], axis = 1)
    print()
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('X', fontsize = 15)
    ax.set_ylabel('Y', fontsize = 15)
    til1 = "HAC ("+str(filePath)+") with "+str(k)+" clusters"
    ax.set_title(til1, fontsize = 20)
    targets = list(set(observed_truth_nparr))
    for pt in targets:
        indicesToKeep = finalDf['Label'] == pt
        ax.scatter(finalDf.loc[indicesToKeep, 'X'], finalDf.loc[indicesToKeep, 'Y'], s = 50)
    ax.legend(targets)
    ax.grid()
    
print("Jaccard Coefficient = " + str(jaccard_coeff))
print("Rand Index = " + str(rand_index))